## Black wall group


#### Задание 
В тестовом задании необходимо произвести анализ транзакций пользователей и найти тех, кто использует более одного аккаунта для совершения транзакций.

Результатом являются id учетных записей разбитые на группы пользователей. 

Т.е. если у одного клиента 5 учетных записей, они должны быть собраны в одну группу.

Ожидается код решения, 
количество получившихся групп клиентов,
файл с разметкой к какой группе принадлежит client_id

В таблице имеются следующие поля:

transaction_id - id транзакции
client_id - id аккаунта (1 пользователь может иметь несколько аккаунтов)
date - дата транзакции
telegram_id - телеграм
wallet_id - банковская карта 
fio_id - ФИО 
email_id - email

___________________________

### РЕШЕНИЕ 

Сначала порассуждаем  какой параметр клиента может быть уникальным: 
    
    client_id - не уникальный ,тк. присвается с каждой новой транзакцией ; 
    telegram_id - не уникальный ,имея несколько тлф ,можно сделать несколько аккаунтов;
    wallet_id - не уникальный, можно иметь несколько банковских карт  на 1 лицо;
    email_id  - не уникальный, можно иметь несколько email на 1 лицо;
    
    fio_id - уникальный , т.к. Каждому человеку присваевается уникальный fio_id
    
       
    В итоге, 
    решение задачи сводится к определению   кол-ва client_id закрепленных  на каждом  fio_id.
    а потом соответсвенно группировка  client_id на каждом fio_id    
    и выгрузка DF.

In [1]:
#Подключаем  библиотеки 

import pandas as pd
from datetime import datetime
import os  

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

sns.set(
font_scale=2,
style="whitegrid",
rc={'figure.figsize':(20,7)}
)

In [2]:
df=pd.read_csv('task.csv')# загружаем датасет 
df.head()

,transaction_id,client_id,date,telegram_id,wallet_id,fio_id,email_id
0,0,302787.0,2020-05-07,6541.0,4537.0,NaN,NaN
1,1,517656.0,2020-05-04,3999.0,NaN,NaN,NaN
2,2,517656.0,2020-05-04,3999.0,NaN,NaN,NaN
3,3,517529.0,2020-05-04,NaN,3164.0,NaN,NaN
4,4,517794.0,2020-05-04,1133.0,5898.0,NaN,NaN


In [3]:
df.shape #оценка  базового размер DF

(75453, 7)

### Первичный анализ  и предобработка данных

In [4]:
df.dtypes # тип данных 

transaction_id      int64
client_id         float64
date               object
telegram_id       float64
wallet_id         float64
fio_id            float64
email_id          float64
dtype: object

In [5]:
df['date'] = pd.to_datetime (df['date']) # необязательно для данного задания т.к. временнные интервалы  не анализируем
# df["fio_id"] = df["fio_id"].fillna(0).astype("int")  # так не можем сделать иначе потеряем транзакцию клиент 0.0 и повесим на него все непонятные транзакции 


In [6]:
df.isna().sum() # оценка пустых значений

transaction_id        0
client_id           109
date                  0
telegram_id       26751
wallet_id         41580
fio_id            74113
email_id          71531
dtype: int64

In [7]:
df = df.drop_duplicates()# убираем дубликаты
df.shape

(75453, 7)

In [8]:
#проверка на кол-во  уникальных  ФИО fio_id
df.fio_id.nunique()

653

In [9]:
df.fio_id.value_counts(dropna = True) # оценка кол-ва групп

518.0    25
404.0    21
235.0    20
449.0    18
614.0    12
         ..
84.0      1
41.0      1
267.0     1
293.0     1
362.0     1
Name: fio_id, Length: 653, dtype: int64

In [22]:
# формируем  группировку пользователей по кол-ву учетных записей client_id
df2= df.groupby(['fio_id','client_id',],as_index=False)\
    .transaction_id.count().sort_values('fio_id',ascending =True) \
    .pivot_table( index='fio_id',values='client_id',aggfunc='count') \
    .rename (columns={'client_id':'GROUPS'} )
df2.head()

,GROUPS
fio_id,
0.0,1
1.0,2
2.0,1
3.0,1
4.0,1


### Промежуточный ответ 1
25 - групп ,т.к. 25  -максимальное кол-во   аккаунтов client_id  на пользователе fio_id =518.0;

653 - Общее кличество пользователей(fio_id)

### Промежуточный ответ 2

client_id учетные записи, разбитые на группы пользователей fio_id (сортировка по fio_id )

In [25]:
# сгруппируем  client_id по  пользователям 
df3= df.groupby(['fio_id','client_id',],as_index=False)\
    .transaction_id.count().sort_values('fio_id',ascending =True)\
    .rename (columns={'transaction_id':'amount_transaction_id'} )
df3.head()

,fio_id,client_id,amount_transaction_id
0,0.0,610745.0,2
1,1.0,610745.0,3
2,1.0,613025.0,1
3,2.0,616998.0,1
4,3.0,615379.0,1


In [26]:
# Вариант 1 - обьеденим базовую таблицу с  таблицей df2 (индекасцией fio_id по группам) , 
#получим базовую таблицу с меткой -client_id принадлежности к группе 
df_itog1=df.merge(df2 ,how='left',on='fio_id')
df_itog1

,transaction_id,client_id,date,telegram_id,wallet_id,fio_id,email_id,GROUPS
0,0,302787.0,2020-05-07,6541.0,4537.0,NaN,NaN,NaN
1,1,517656.0,2020-05-04,3999.0,NaN,NaN,NaN,NaN
2,2,517656.0,2020-05-04,3999.0,NaN,NaN,NaN,NaN
3,3,517529.0,2020-05-04,NaN,3164.0,NaN,NaN,NaN
4,4,517794.0,2020-05-04,1133.0,5898.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
75448,75448,566286.0,2020-12-26,7264.0,NaN,NaN,NaN,NaN
75449,75449,603162.0,2020-12-26,2536.0,NaN,NaN,NaN,NaN
75450,75450,613962.0,2020-12-26,NaN,NaN,NaN,NaN,NaN
75451,75451,569724.0,2020-12-26,1184.0,NaN,NaN,NaN,NaN


In [15]:
#  Вариант 2- На мой взгляд, лучше отсечь NaN  и обьеденить  базовую таблицу с  таблицей df2 (индекасцией fio_id по группам) 
# по совпадающим значениям -inner , получим базовую таблицу с меткой -client_id принадлежности к группе 
df_itog2=df.merge(df2 ,how='inner',on='fio_id')
df_itog2

,transaction_id,client_id,date,telegram_id,wallet_id,fio_id,email_id,GROUPS
0,7521,605526.0,2020-11-27,42.0,4817.0,580.0,NaN,1
1,7601,605920.0,2020-11-28,163.0,5298.0,271.0,NaN,1
2,7602,605903.0,2020-11-28,7155.0,3052.0,523.0,NaN,1
3,7636,600919.0,2020-11-28,4916.0,4431.0,510.0,NaN,2
4,26861,609357.0,2020-12-12,4916.0,4431.0,510.0,NaN,2
...,...,...,...,...,...,...,...,...
1335,65465,574542.0,2020-12-26,6826.0,6490.0,374.0,NaN,1
1336,65469,574542.0,2020-12-26,6826.0,6490.0,374.0,NaN,1
1337,65463,617589.0,2020-12-26,NaN,4355.0,122.0,NaN,1
1338,65470,617709.0,2020-12-26,NaN,3933.0,465.0,NaN,1


 ### Итоговый результат 3
 
 В задании указано  -  получить файл с разметкой к какой группе принадлежит client_id 
 
поэтому оставлю  на выгрузку - вариант 1.  


In [17]:
#  выгрузим DF в папку  Мои документы  
os.makedirs('C:/Users/User/Documents', exist_ok=True)  
df_itog1.to_csv('C:/Users/User/Documents/df_itog1.csv')